In [1]:
import json
import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import (udf, col)

from google.cloud import storage

In [13]:
BUCKET = "rjr-portal-da-transparencia"
YEAR = 2020
REGION = "nordeste"

In [14]:
def load_json(name):
    bucket = storage.Client().get_bucket(BUCKET)
    blob = bucket.blob(f'aux_tables/{name}.json')
    maps = json.loads(blob.download_as_string())
    return maps


def mapping(df, map_, column, type_return):
    map_func = udf(lambda key: map_.get(str(key)), type_return())
    df = df.withColumn(column, map_func(col(column)))
    return df

def string_to_date(df, column, year):
    if year > 2014:
        pattern = '%d/%m/%Y'
    else:
        pattern = "%d%b%Y:%H:%M:%S"
    map_func =  udf (lambda date: datetime.strptime(date, pattern) 
                     if type(date) == str 
                     else None, DateType())
    df = df.withColumn(column, map_func(col(column)))
    return df

In [15]:
spark = SparkSession.builder.appName("censo").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [16]:
schema = load_json("matriculas_schema")
try:
    schema = StructType.fromJson(schema)
except:
    schema = StructType.fromJson(json.loads(schema))
df = spark \
    .read \
    .options(header=True, delimiter="|", encoding="utf8") \
    .schema(schema=schema) \
    .csv(f"gs://{BUCKET}/landing_zone/censo-escolar/{YEAR}/matricula_{REGION}.csv" )

In [17]:
maps = {**load_json("matriculas_maps"), **load_json("regioes_maps")}
string_columns = [column for column in df.columns 
                  if column.startswith("TP") or column.startswith("CO")]

for column in string_columns:
    if column in maps:
        df = mapping(df, maps[column], column, StringType)

In [18]:
bool_columns = [column for column in df.columns 
                  if column.startswith("IN")]

mapping_bool = {
    "0": False,
    "1": True
}

for column in bool_columns:
    df = mapping(df, mapping_bool, column, BooleanType)

In [19]:
drops = ['CO_CURSO_EDUC_PROFISSIONAL',
 'TP_MEDIACAO_DIDATICO_PEDAGO',
 'TP_UNIFICADA',
 'TP_TIPO_ATENDIMENTO_TURMA',
 'TP_TIPO_LOCAL_TURMA',
 'CO_ENTIDADE',
 'CO_DISTRITO',
 'TP_DEPENDENCIA',
 'TP_CATEGORIA_ESCOLA_PRIVADA',
 'TP_CONVENIO_PODER_PUBLICO',
 'TP_REGULAMENTACAO']

In [20]:
df = df.drop(*drops)

In [21]:
df

NU_ANO_CENSO,ID_ALUNO,ID_MATRICULA,NU_MES,NU_ANO,NU_IDADE_REFERENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,CO_PAIS_ORIGEM,CO_UF_NASC,CO_MUNICIPIO_NASC,CO_UF_END,CO_MUNICIPIO_END,TP_ZONA_RESIDENCIAL,TP_LOCAL_RESID_DIFERENCIADA,IN_NECESSIDADE_ESPECIAL,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_DEF_AUDITIVA,IN_DEF_FISICA,IN_DEF_INTELECTUAL,IN_SURDEZ,IN_SURDOCEGUEIRA,IN_DEF_MULTIPLA,IN_AUTISMO,IN_SUPERDOTACAO,IN_RECURSO_LEDOR,IN_RECURSO_TRANSCRICAO,IN_RECURSO_INTERPRETE,IN_RECURSO_LIBRAS,IN_RECURSO_LABIAL,IN_RECURSO_AMPLIADA_18,IN_RECURSO_AMPLIADA_24,IN_RECURSO_CD_AUDIO,IN_RECURSO_PROVA_PORTUGUES,IN_RECURSO_VIDEO_LIBRAS,IN_RECURSO_BRAILLE,IN_RECURSO_NENHUM,IN_AEE_LIBRAS,IN_AEE_LINGUA_PORTUGUESA,IN_AEE_INFORMATICA_ACESSIVEL,IN_AEE_BRAILLE,IN_AEE_CAA,IN_AEE_SOROBAN,IN_AEE_VIDA_AUTONOMA,IN_AEE_OPTICOS_NAO_OPTICOS,IN_AEE_ENRIQ_CURRICULAR,IN_AEE_DESEN_COGNITIVO,IN_AEE_MOBILIDADE,TP_OUTRO_LOCAL_AULA,IN_TRANSPORTE_PUBLICO,TP_RESPONSAVEL_TRANSPORTE,IN_TRANSP_BICICLETA,IN_TRANSP_MICRO_ONIBUS,IN_TRANSP_ONIBUS,IN_TRANSP_TR_ANIMAL,IN_TRANSP_VANS_KOMBI,IN_TRANSP_OUTRO_VEICULO,IN_TRANSP_EMBAR_ATE5,IN_TRANSP_EMBAR_5A15,IN_TRANSP_EMBAR_15A35,IN_TRANSP_EMBAR_35,TP_ETAPA_ENSINO,IN_ESPECIAL_EXCLUSIVA,IN_REGULAR,IN_EJA,IN_PROFISSIONALIZANTE,ID_TURMA,NU_DURACAO_TURMA,NU_DUR_ATIV_COMP_MESMA_REDE,NU_DUR_ATIV_COMP_OUTRAS_REDES,NU_DUR_AEE_MESMA_REDE,NU_DUR_AEE_OUTRAS_REDES,NU_DIAS_ATIVIDADE,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,TP_LOCALIZACAO,IN_CONVENIADA_PP,IN_MANT_ESCOLA_PRIVADA_EMP,IN_MANT_ESCOLA_PRIVADA_ONG,IN_MANT_ESCOLA_PRIVADA_OSCIP,IN_MANT_ESCOLA_PRIV_ONG_OSCIP,IN_MANT_ESCOLA_PRIVADA_SIND,IN_MANT_ESCOLA_PRIVADA_SIST_S,IN_MANT_ESCOLA_PRIVADA_S_FINS,TP_LOCALIZACAO_DIFERENCIADA,IN_EDUCACAO_INDIGENA
2020,DB40542357729209B...,101,4,2010,10,10,Feminino,Parda,Brasileira,BRASIL,Alagoas,Maceió,Alagoas,Maceió,Urbana,null,false,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Não recebe escola...,false,null,null,null,null,null,null,null,null,null,null,null,Ensino Fundamenta...,false,true,false,false,6647419,270,0,0,0,0,5,Nordeste,Leste Alagoano,Maceió,Alagoas,Maceió,Urbana,false,true,false,false,false,false,false,false,A escola não está...,false
2020,56A624969D3381901...,104,8,2009,10,11,Feminino,Branca,Brasileira,BRASIL,Alagoas,Maceió,Alagoas,Maceió,Urbana,null,false,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Não recebe escola...,false,null,null,null,null,null,null,null,null,null,null,null,Ensino Fundamenta...,false,true,false,false,6647419,270,0,0,0,0,5,Nordeste,Leste Alagoano,Maceió,Alagoas,Maceió,Urbana,false,true,false,false,false,false,false,false,A escola não está...,false
2020,36C598A929F3B6EB6...,106,3,2010,10,10,Feminino,Parda,Brasileira,BRASIL,Alagoas,Maceió,Alagoas,Maceió,Urbana,null,false,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Não recebe escola...,false,null,null,null,null,null,null,null,null,null,null,null,Ensino Fundamenta...,false,true,false,false,6647419,270,0,0,0,0,5,Nordeste,Leste Alagoano,Maceió,Alagoas,Maceió,Urbana,false,true,false,false,false,false,false,false,A escola não está...,false
2020,CE136BD21BAF3A0D0...,109,8,2009,10,11,Masculino,Parda,Brasileira,BRASIL,Alagoas,Maceió,Alagoas,Maceió,Urbana,null,false,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Não recebe escola...,false,null,null,null,null,null,null,null,null,null,null,null,Ensino Fundamenta...,false,true,false,false,6647419,270,0,0,0,0,5,Nordeste,Leste Alagoano,Maceió,Alagoas,Maceió,Urbana,false,true,false,false,false,false,false,false,A escola não está...,false
2020,88D4C9CC4EC75E0

In [23]:
df.write.parquet(f"censo-escolar/{YEAR}/matriculas_{REGION}.parquet",
                compression="uncompressed")

In [24]:
df.write.parquet(f"gs://rjr-portal-da-transparencia/temp/censo-escolar/{YEAR}/matriculas_{REGION}.parquet",
                compression="snappy")